In [105]:
import music21 as m
import torch
import torch.nn as nn
import torch.optim as optim
from collections import Counter
import numpy as np
import random

In [45]:
songs = m.converter.parse('./data/nottingham_database/reelsA-C.abc')

In [47]:
##### how to extract melodies in abc format ######

print(a[0])
len(a.elements) # 'reelsA-C.abc' 안에 81곡 포함 (abc파일은 보통 한 파일 안에 여러 곡이 들어있음)
a[0].elements # Elements of the 1st score
#print(a[0].flat.notes.stream().show('text')) # notes and chordsymbol of 1st score
print(a[0].flat.getElementsByClass('Note').pitches) # extract melodies in 1st score

<music21.stream.Score 0x11b5d7b50>
[<music21.pitch.Pitch A5>, <music21.pitch.Pitch F#5>, <music21.pitch.Pitch G5>, <music21.pitch.Pitch F#5>, <music21.pitch.Pitch E5>, <music21.pitch.Pitch D5>, <music21.pitch.Pitch C#5>, <music21.pitch.Pitch B4>, <music21.pitch.Pitch A4>, <music21.pitch.Pitch F#5>, <music21.pitch.Pitch G5>, <music21.pitch.Pitch B4>, <music21.pitch.Pitch C#5>, <music21.pitch.Pitch B4>, <music21.pitch.Pitch C#5>, <music21.pitch.Pitch D5>, <music21.pitch.Pitch A4>, <music21.pitch.Pitch G4>, <music21.pitch.Pitch F#4>, <music21.pitch.Pitch E4>, <music21.pitch.Pitch F#4>, <music21.pitch.Pitch A4>, <music21.pitch.Pitch A4>, <music21.pitch.Pitch G4>, <music21.pitch.Pitch F#4>, <music21.pitch.Pitch G4>, <music21.pitch.Pitch B4>, <music21.pitch.Pitch B4>, <music21.pitch.Pitch B4>, <music21.pitch.Pitch C#5>, <music21.pitch.Pitch D5>, <music21.pitch.Pitch E5>, <music21.pitch.Pitch C#5>, <music21.pitch.Pitch D5>, <music21.pitch.Pitch E5>, <music21.pitch.Pitch C#5>, <music21.pitch.P

In [63]:
all_songs_notes = [song.flat.getElementsByClass('Note').pitches for song in songs]
notes = [[pitch.nameWithOctave for pitch in song_notes] for song_notes in all_songs_notes]
print(notes[0]) # melody notes of 1st score 
unique_notes = list(set(sum(notes, [])))
print(len(unique_notes)) # number of unique_notes in all 81 scores in reelA-C.abc file: 30
print(len(sum(notes, []))) # number of total notes in all 81 scores in reelA-C.abc file: 7813

['A5', 'F#5', 'G5', 'F#5', 'E5', 'D5', 'C#5', 'B4', 'A4', 'F#5', 'G5', 'B4', 'C#5', 'B4', 'C#5', 'D5', 'A4', 'G4', 'F#4', 'E4', 'F#4', 'A4', 'A4', 'G4', 'F#4', 'G4', 'B4', 'B4', 'B4', 'C#5', 'D5', 'E5', 'C#5', 'D5', 'E5', 'C#5', 'B4', 'C#5', 'A4', 'D5', 'E5', 'F#5', 'E5', 'F#5', 'D5', 'E5', 'F#5', 'G5', 'F#5', 'G5', 'E5', 'F#5', 'G5', 'A5', 'B4', 'C#5', 'B4', 'C#5', 'D5', 'D5']
30


In [67]:
note_freq = dict(Counter(sum(notes, [])))
note_freq = sorted(note_freq.items(), key=lambda x: x[1], reverse=True)
idx2note = [item[0] for item in note_freq]
note2idx = {item[0]:note_freq.index(item) for item in note_freq}
print(idx2note)
print(note2idx)

['D5', 'A4', 'B4', 'E5', 'G4', 'F#5', 'C#5', 'C5', 'G5', 'F#4', 'A5', 'E4', 'D4', 'G#4', 'B5', 'G#5', 'F5', 'F4', 'B-4', 'C#4', 'C4', 'A3', 'E#5', 'B3', 'D#5', 'B-5', 'A#4', 'G3', 'D-5', 'F-5']
{'D5': 0, 'A4': 1, 'B4': 2, 'E5': 3, 'G4': 4, 'F#5': 5, 'C#5': 6, 'C5': 7, 'G5': 8, 'F#4': 9, 'A5': 10, 'E4': 11, 'D4': 12, 'G#4': 13, 'B5': 14, 'G#5': 15, 'F5': 16, 'F4': 17, 'B-4': 18, 'C#4': 19, 'C4': 20, 'A3': 21, 'E#5': 22, 'B3': 23, 'D#5': 24, 'B-5': 25, 'A#4': 26, 'G3': 27, 'D-5': 28, 'F-5': 29}


In [71]:
idx2char = ['e','t','a','o','i','n','s','r','h','l','d','c','u','m','g','f','p',
            'w','y','b',',','.','v','k','-','x','0','j','q','z'] # source: https://mdickens.me/typing/letter_frequency.html
char2idx = {ch:idx2char.index(ch) for ch in idx2char}
print(idx2char)
print(char2idx)


['e', 't', 'a', 'o', 'i', 'n', 's', 'r', 'h', 'l', 'd', 'c', 'u', 'm', 'g', 'f', 'p', 'w', 'y', 'b', ',', '.', 'v', 'k', '-', 'x', '0', 'j', 'q', 'z']
{'e': 0, 't': 1, 'a': 2, 'o': 3, 'i': 4, 'n': 5, 's': 6, 'r': 7, 'h': 8, 'l': 9, 'd': 10, 'c': 11, 'u': 12, 'm': 13, 'g': 14, 'f': 15, 'p': 16, 'w': 17, 'y': 18, 'b': 19, ',': 20, '.': 21, 'v': 22, 'k': 23, '-': 24, 'x': 25, '0': 26, 'j': 27, 'q': 28, 'z': 29}


In [227]:
lr = 0.001
batch_size = 100
input_size = 30
hidden_size = 10
output_size = 30
seq_length = 4
num_layers = 2
num_directions = 2

In [141]:
one_hot_lookup = np.eye(len(unique_notes))
one_hot_note = [one_hot_lookup[note2idx[note]] for note in sum(notes, [])]
X = [one_hot_note[i:i+4] for i in range(len(one_hot_note)-4)]
Y = [one_hot_note[i+4] for i in range(len(one_hot_note)-4)]
Y = [list(one_hot).index(1) for one_hot in Y]

print(len(X)) # 7809 
print(len(Y)) # 7809 total training samples
print(X[:2])
print(Y[:2])

7809
7809
[[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])], [array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 1., 0., 0., 0.,

In [144]:
def make_batch(X, Y, batch_size):
    X_ = X.copy()
    Y_ = Y.copy()
    samples = list(zip(X_,Y_))
    random.shuffle(samples)
    batch = random.sample(samples, batch_size)
    [input_batch, target_batch] = list(zip(*batch))
    input_batch = torch.tensor(input_batch, dtype=torch.float32)
    target_batch = torch.tensor(target_batch, dtype=torch.int64)
    return input_batch, target_batch

In [236]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.batch_size = batch_size
        self.input_size = input_size 
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.seq_length = seq_length
        self.num_layers = num_layers
        self.num_directions = num_directions
        
        self.rnn = nn.LSTM(self.input_size, self.hidden_size, num_layers=self.num_layers, bidirectional=True, 
                          batch_first=True)
        self.fc1 = nn.Linear(self.hidden_size, self.output_size, bias=True)
        self.fc2 = nn.Linear(self.num_directions*self.num_layers*self.output_size, self.output_size, bias=True)
        
    def init_hidden(self):
        hidden = torch.zeros(self.num_layers*self.num_directions, self.batch_size, self.hidden_size)
        return hidden
    
    def forward(self, X, hidden):
        X = X.view(-1, self.seq_length, self.input_size)
        outputs, (hidden, cell) = self.rnn(X, hidden)
        #print("rnn outputs: ", outputs.size(), "rnn last hidden: ", hidden.size())
        fc1_output = self.fc1(hidden)
        #print("fc1 output: ", fc1_output.size())
        fc1_output = torch.transpose(fc1_output, 0,1)
        fc1_output = fc1_output.reshape(-1, self.num_directions*self.num_layers*self.output_size)
        fc2_output = self.fc2(fc1_output)
        #print("fc2 output: ", fc2_output.size())
        return fc2_output

In [237]:
model = Model()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

for epoch in range(1000):
    input_batch, target_batch = make_batch(X,Y,batch_size)
    #print("input_batch: ", input_batch.size(), "target_batch: ", target_batch.size())
    optimizer.zero_grad()
    
    hidden = model.init_hidden()
    cell = model.init_hidden()
    pred = model.forward(input_batch, (hidden,cell))
    #print(pred.size(), target_batch.size())
    pred = pred.view(-1, model.output_size)
    target_batch = target_batch.view(model.batch_size)
    
    loss = criterion(pred, target_batch)
    
    value, idx = pred.data.max(dim=1)
    predicted = [idx2note[x] for x in idx.numpy()]
    
    if (epoch+1) % 100 == 0:
        
        batch_note = []
        for input_ in input_batch:
            one_batch_notes = [idx2note[list(note).index(1)] for note in input_]
            batch_note.append(one_batch_notes)
            
        #print("epoch: %d | loss: %1.3f" % (epoch+1, loss), "| pred: ", list(zip(batch_note, predicted)))
        print("epoch: %d | loss: %1.3f" % (epoch+1, loss))
        print("input_batch: ", input_batch.size(), "pred: ", pred.size(), "target_batch: ", target_batch.size())
    loss.backward()
    optimizer.step()


epoch: 100 | loss: 2.527
input_batch:  torch.Size([100, 4, 30]) pred:  torch.Size([100, 30]) target_batch:  torch.Size([100])
epoch: 200 | loss: 2.251
input_batch:  torch.Size([100, 4, 30]) pred:  torch.Size([100, 30]) target_batch:  torch.Size([100])
epoch: 300 | loss: 2.163
input_batch:  torch.Size([100, 4, 30]) pred:  torch.Size([100, 30]) target_batch:  torch.Size([100])
epoch: 400 | loss: 2.029
input_batch:  torch.Size([100, 4, 30]) pred:  torch.Size([100, 30]) target_batch:  torch.Size([100])
epoch: 500 | loss: 2.011
input_batch:  torch.Size([100, 4, 30]) pred:  torch.Size([100, 30]) target_batch:  torch.Size([100])
epoch: 600 | loss: 2.004
input_batch:  torch.Size([100, 4, 30]) pred:  torch.Size([100, 30]) target_batch:  torch.Size([100])
epoch: 700 | loss: 1.871
input_batch:  torch.Size([100, 4, 30]) pred:  torch.Size([100, 30]) target_batch:  torch.Size([100])
epoch: 800 | loss: 1.965
input_batch:  torch.Size([100, 4, 30]) pred:  torch.Size([100, 30]) target_batch:  torch.Size

In [250]:
input_seed = ['h','e','l','l']
input_one_hot = [char2idx[ch] for ch in input_seed]
input_one_hot = [one_hot_lookup[x] for x in input_one_hot]
generation = input_one_hot.copy()
input_one_hot_tensor = torch.tensor([input_one_hot], dtype=torch.float32)
#print(input_one_hot_tensor.size())

generation_time = 50

for i in range(generation_time):
    test_hidden = torch.randn(model.num_layers*model.num_directions, 1, model.hidden_size)
    test_cell = torch.randn(model.num_layers*model.num_directions, 1, model.hidden_size)
    char_output = model.forward(input_one_hot_tensor, (test_hidden, test_cell))
    char_output = char_output.view(-1, model.output_size)
    #print(char_output, char_output.size())
    
    value, idx = char_output.data.max(dim=1)
    next_input = one_hot_lookup[int(idx.squeeze())]
    generation.append(next_input)
    input_one_hot_tensor = generation[-4:]
    input_one_hot_tensor = torch.tensor([input_one_hot_tensor], dtype=torch.float32)
    #print(input_one_hot_tensor.size())
    
predicted_char = [idx2char[list(one_hot).index(1)] for one_hot in generation]
print('predicted: ', ''.join(predicted_char))

predicted:  hellittatithnoeoetatiearatiarhlhdithdheoesteotttrtilie


In [170]:
a = torch.tensor([[[1,2,3],[0.1,0.2,0.3]],[[4,5,6],[0.4,0.5,0.6]]])
print(a)
print(a.size())
print(a.size()[2])
b = torch.transpose(a, 0,1)
print(b)
print(b.reshape(2,-1))

tensor([[[1.0000, 2.0000, 3.0000],
         [0.1000, 0.2000, 0.3000]],

        [[4.0000, 5.0000, 6.0000],
         [0.4000, 0.5000, 0.6000]]])
torch.Size([2, 2, 3])
3
tensor([[[1.0000, 2.0000, 3.0000],
         [4.0000, 5.0000, 6.0000]],

        [[0.1000, 0.2000, 0.3000],
         [0.4000, 0.5000, 0.6000]]])
tensor([[1.0000, 2.0000, 3.0000, 4.0000, 5.0000, 6.0000],
        [0.1000, 0.2000, 0.3000, 0.4000, 0.5000, 0.6000]])
